In [1]:
import boto3

In [2]:
session = boto3.session.Session(profile_name='prod')
s3 = session.client('s3')
s3.list_buckets()['Buckets']

[{'Name': 'aws-athena-query-results-339782887627-eu-west-2',
  'CreationDate': datetime.datetime(2019, 11, 14, 11, 29, 25, tzinfo=tzlocal())},
 {'Name': 'mi-databricks-prod',
  'CreationDate': datetime.datetime(2019, 11, 14, 11, 31, 13, tzinfo=tzlocal())},
 {'Name': 'mi-datalake-prod',
  'CreationDate': datetime.datetime(2020, 1, 20, 12, 35, 17, tzinfo=tzlocal())},
 {'Name': 'mi-experian-prod',
  'CreationDate': datetime.datetime(2019, 11, 26, 11, 28, 19, tzinfo=tzlocal())},
 {'Name': 'mi-glue-etl-app-prod',
  'CreationDate': datetime.datetime(2020, 1, 17, 14, 16, 26, tzinfo=tzlocal())},
 {'Name': 'mi-glue-etl-sql-queries-prod',
  'CreationDate': datetime.datetime(2020, 1, 17, 14, 16, 26, tzinfo=tzlocal())},
 {'Name': 'mi-lambda-deployments-prod',
  'CreationDate': datetime.datetime(2020, 1, 17, 14, 16, 26, tzinfo=tzlocal())},
 {'Name': 'mi-pickles-prod',
  'CreationDate': datetime.datetime(2019, 11, 14, 11, 32, 32, tzinfo=tzlocal())},
 {'Name': 'mi-python-libraries-prod',
  'CreationD

In [3]:
from configparser import ConfigParser

config_object = ConfigParser()
config_object.read("/home/jovyan/.aws/credentials")
profile_info = config_object["prod"]

In [4]:
from pyspark import SparkContext, SparkConf, SQLContext

conf = (
    SparkConf()
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.access.key", profile_info.get('aws_access_key_id'))
    .set("spark.hadoop.fs.s3a.secret.key", profile_info.get('aws_secret_access_key'))
    .set("spark.hadoop.fs.s3a.endpoint", f"s3-{profile_info.get('region')}.amazonaws.com")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("com.amazonaws.services.s3.enableV4", True)
    .set("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true")
)
sc = SparkContext(conf=conf).getOrCreate()

sqlContext = SQLContext(sc)
df = sqlContext.read.parquet("s3a://mi-datalake-prod/warehouse/platform/creditsafe_company_list/")

In [5]:
df

DataFrame[creditsafe_id: string, company_number: string, company_name: string, incorporation_date: date, charity_number: string, status_code: string, status_description: string, liquidation_status: string, administration_order_flag: string, in_administration_flag: string, voluntary_agreement_flag: string, receiver_manager_flag: string, company_type: string, company_type_description: string, company_sic: string, accounts_latest_date: date, accounts_due_date: date, accounts_made_up_to_date: date, credit_rating: string, postcode: string, parent_company_number: string, ultimate_parent_company_number: string, no_of_employees: string]